# Chapter 1. Probability
[Link to chapter online](https://allendowney.github.io/ThinkBayes2/chap01.html)

## Warning

The content of this file may be incorrect, erroneous and/or harmful. Use it at Your own risk.

## Imports

In [ ]:
import CSV as Csv
import DataFrames as Dfs
import Statistics as Stats

## Linda the Banker

> Linda is 31 years old, single, outspoken, and very bright. She majored in philosophy. As a student, she was deeply concerned with issues of discrimination and social justice, and also participated in anti-nuclear demonstrations. Which is more probable?
> 1. Linda is a bank teller.
> 2. Linda is a bank teller and is active in the feminist movement.

## Probability

[Link to the data file (gss_bayes.csv) online](https://raw.githubusercontent.com/AllenDowney/BiteSizeBayes/master/gss_bayes.csv)

In [ ]:
gss = Dfs.DataFrame(Csv.File("./gss_bayes.csv"))
first(gss, 5)

## Fraction of bankers

The code for “Banking and related activities” is 6870, so we can select bankers like this:

In [ ]:
banker = (gss.indus10 .== 6870)
first(banker, 5)

In [ ]:
sum(banker)

In [ ]:
Stats.mean(banker)

## The probability function

In [ ]:
function getProb(v::BitVector)::Float64
    return Stats.mean(v)
end

In [ ]:
getProb(banker)

columng `gss.sex` coding:
1. male
2. female

In [ ]:
female = (gss.sex .== 2);

In [ ]:
getProb(female)

## Political Views and Parties

`polviews` coding:
1. Extremely liberal
2. Liberal
3. Slightly liberal
4. Moderate
5. Slightly conservative
6. Conservative
7. Extremely conservative

In [ ]:
liberal = (gss.polviews .<= 3);

In [ ]:
getProb(liberal)

`partyid` coding:
0. Strong democrat
1. Not strong democrat
2. Independent, near democrat
3. Independent
4. Independent, near republican
5. Not strong republican
6. Strong republican
7. Other party

In [ ]:
democrat = (gss.partyid .<= 1);

In [ ]:
getProb(democrat)